In [ ]:
# so will label user as poly and mono based on the tweets
# will further classify poly into 
    # talk about drug after their first mention of juul
    # talk d before their mention of juul

In [ ]:
import pandas as pd
import numpy as np
import tweepy
from matplotlib import pyplot as plt
from tqdm import tqdm
import pickle
import ast

In [ ]:
import gensim
import nltk
import pandas as pd
import numpy as np
import tweepy
from matplotlib import pyplot as plt
from tqdm import tqdm

In [ ]:
pd.set_option('display.max_colwidth', -1)

In [ ]:
# need to change with 3200 limit data
df_timeline = pd.read_csv("/home/akshayuppal3/userTimelineData.csv", lineterminator="\n")

In [ ]:
a =  df_timeline.groupby("userID")['tweetText'].agg("count")

In [ ]:
users = list(a.index)

In [ ]:
plt.plot(np.arange(0,len(a)),a.values)
plt.show()

In [ ]:
df_user1 = df_timeline[df_timeline.userID == users[0]]


In [ ]:
weed_words.remove('gage')

In [ ]:
weed_words.remove('skunk')

# need watson classifier for categorical classification

In [ ]:
# next user watson to classify text into categories
df_user1.loc[df_user1.tweetText.str.contains(pattern)]

In [ ]:
poly_user = list()
for user in tqdm(users):
    df_temp = df_timeline[df_timeline.userID == user]
    if(len(df_temp.loc[df_temp.tweetText.str.contains(pattern)]) > 0):
        poly_user.append(user)

In [ ]:
ex = df_timeline.loc[df_timeline.userID == poly_user[1]]
ex.loc[ex.tweetText.str.contains(pattern)]

In [ ]:
# so will create embeddings for our tweets and manually label 500 of the tweet containing weeds and classify them 
# them as substance abouse or not

In [ ]:
len(df_timeline.tweetText)

In [ ]:
## loading the weed words or other form of drugs (top 30 mentioned at https://www.powerthesaurus.org/weed/synonyms)
weed_words = pd.read_excel("/Users/akshayuppal/Desktop/thesis/backup_stuff/Weed_words.xlsx",header=None)
weed_words.columns = ['weedID']
weed_words = list(weed_words.weedID)
weed_words.remove('gage')
weed_words.remove('skunk')
# weed_words.remove('grass')
# weed_words.remove('pot')
weed_words.remove("dope")
# weed_words.remove("joint")
# weed_words.remove("herb")
# weed_words.remove("bud")

In [ ]:
# dump the updated weed words in models directory
with open("/Users/akshayuppal/Desktop/thesis/twitter_juul/models/weed_words.pkl","wb") as f:
    pickle.dump(weed_words,f)

In [ ]:
weed_words = [(" " + word+" ") for word in weed_words]
pattern = "|".join(weed_words)

In [179]:
pattern

' weed | ganja | marijuana | grass | cannabis | pot | smoke | mary jane | hemp | marihuana | hash | reefer | hashish | herb | bhang | green goddess | locoweed | maryjane | bud | spliff | wacky baccy | joint | sinsemilla | doobie | tobacco | acapulco gold '

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')

### tokenize the sentences for the gensim.. we are going to classify the  sentences as weeds or not.. manual label(500 sent)

In [198]:
from gensim.models import Word2Vec
from sklearn.decomposition import PCA
from matplotlib import pyplot
tqdm.pandas()

### extracting the data

In [634]:
df_timeline = pd.read_csv("~/Desktop/thesis/backup_stuff/userTimelineData.csv",lineterminator="\n")

In [635]:
# keeping only english tweets
df_timeline = (df_timeline[df_timeline.lang == 'en'])

In [636]:
# filter those that contain the occurance of weed or similar words
df_tweet_weeds = df_timeline[df_timeline['tweetText'].str.contains(pattern, case = False)]

In [637]:
df_nonweed = (df_timeline.loc[~df_timeline.index.isin(df_tweet_weeds.index)])

In [638]:
# taking sample of size weed tweets
df_nonweed =(df_nonweed.sample(frac=1))
nonweed_sample = df_nonweed[:len(df_tweet_weeds)]

In [639]:
# randomize the data before creating embedding 
df_tweet_weeds =(df_tweet_weeds.sample(frac=1))

In [640]:
weed_sample = (df_tweet_weeds)

In [641]:
mix_sample = weed_sample.append(nonweed_sample)

In [642]:
#randomize weed and non weed tweets
mix_sample = (mix_sample.sample(frac=1))

In [645]:
len(mix_sample)

48622

In [295]:
import nltk
# for df
def get_tokenize_words(text):
    tkz = nltk.tokenize
    words = tkz.word_tokenize(text)
    words = [ele for ele in words if ((ele not in stopwords) and len(ele)>2 and (ele.isalpha()))]
    words = [word.lower() for word in words]
    return words

In [296]:
def hashtag_count(df):
    if df is not np.nan:
        hashtags = ast.literal_eval(df)
        if (hashtags is not None):
            return (len(hashtags))
        else:
            return (0)
    else:
        return (0)

In [ ]:
def count_words(sentence):
    words = get_tokenize_words(sentence)
    return len(words)

In [454]:
def get_words(sentence):
    words = get_tokenize_words(sentence)
    return (words)

In [513]:
# @param dataframe
def get_sentences(df,column):
    sentences = list(df[column].progress_apply(get_words))
    return(sentences)

In [652]:
# @param dataframe --slow not , using above
def get_sentences1(df,column):
    if column in df:
        sentences = list()
        for index,row in tqdm(df.iterrows(),total=len(df)):
            words = get_words(row[column])
            sentences.append(words)
        return(sentences)

In [338]:
## adding further features for the tweets
def create_features(df):
    df_new = pd.DataFrame([])
    if ('tweetText' in df.columns):
        df_new['no_urls'] = df['tweetText'].str.count(r'(https?://\S+)')
        df_new['no_authors'] = df['tweetText'].str.count(r'(\@\w+)')
        if ('hashtags' in df.columns):
            df_new['no_hashtags'] = df['hashtags'].apply(hashtag_count)
        # modifying the text after we got the features
        tweet_texts = pd.DataFrame(df['tweetText'].str.replace(r'(https?://\S+)',""))  # urls
        tweet_texts = pd.DataFrame(df['tweetText'].str.replace(r'(\@\w+)',"author"))  # author mentions
        tweet_texts = pd.DataFrame(df['tweetText'].str.replace(r'(\#\w+)',""))         # removing hashtags
        df_new['tweetText'] = tweet_texts
        df_new['no_words'] = df['tweetText'].progress_apply(count_words)
        return(df_new)

### Extracting features

In [647]:
## getting other features
mix_sample['no_urls'] = mix_sample['tweetText'].str.count(r'(https?://\S+)')
mix_sample['no_authors'] = mix_sample['tweetText'].str.count(r'(\@\w+)')
mix_sample['no_hashtags'] = mix_sample['hashtags'].apply(hashtag_count)

In [648]:
# modifying the text after we got the features
tweet_texts = pd.DataFrame(mix_sample['tweetText'].str.replace(r'(https?://\S+)',""))  # urls
tweet_texts = pd.DataFrame(tweet_texts['tweetText'].str.replace(r'(\@\w+)',"author"))  # author mentions
tweet_texts = pd.DataFrame(tweet_texts['tweetText'].str.replace(r'(\#\w+)',""))         # removing hashtags

In [649]:
mix_sample['tweetText'] = tweet_texts

In [650]:
# count words after we remove all of the other things(urls, authors, hashtags) # replace if find better
mix_sample['no_words'] = mix_sample['tweetText'].progress_apply(count_words)








  0%|          | 0/48622 [00:00<?, ?it/s]






  0%|          | 207/48622 [00:00<00:23, 2060.85it/s]






  1%|          | 439/48622 [00:00<00:22, 2129.85it/s]






  1%|▏         | 692/48622 [00:00<00:21, 2235.80it/s]






  2%|▏         | 950/48622 [00:00<00:20, 2326.07it/s]






  2%|▏         | 1190/48622 [00:00<00:20, 2346.42it/s]






  3%|▎         | 1444/48622 [00:00<00:19, 2397.85it/s]






  3%|▎         | 1685/48622 [00:00<00:19, 2399.89it/s]






  4%|▍         | 1910/48622 [00:00<00:19, 2352.04it/s]






  4%|▍         | 2135/48622 [00:00<00:20, 2289.37it/s]






  5%|▍         | 2372/48622 [00:01<00:20, 2311.74it/s]






  5%|▌         | 2608/48622 [00:01<00:19, 2319.06it/s]






  6%|▌         | 2837/48622 [00:01<00:20, 2267.19it/s]






  6%|▋         | 3076/48622 [00:01<00:19, 2302.45it/s]






  7%|▋         | 3305/48622 [00:01<00:19, 2268.91it/s]






  7%|▋         | 3537/48622 [00:01<00:19, 2281.49it/s]






  8%|▊         | 3765/48622 [00:01

 49%|████▊     | 23660/48622 [00:10<00:10, 2276.04it/s]






 49%|████▉     | 23891/48622 [00:10<00:10, 2283.34it/s]






 50%|████▉     | 24120/48622 [00:10<00:10, 2275.62it/s]






 50%|█████     | 24348/48622 [00:10<00:10, 2256.52it/s]






 51%|█████     | 24588/48622 [00:10<00:10, 2294.11it/s]






 51%|█████     | 24825/48622 [00:10<00:10, 2312.74it/s]






 52%|█████▏    | 25057/48622 [00:10<00:10, 2300.88it/s]






 52%|█████▏    | 25288/48622 [00:10<00:10, 2274.95it/s]






 52%|█████▏    | 25516/48622 [00:10<00:10, 2258.61it/s]






 53%|█████▎    | 25745/48622 [00:11<00:10, 2266.85it/s]






 53%|█████▎    | 25973/48622 [00:11<00:09, 2269.97it/s]






 54%|█████▍    | 26201/48622 [00:11<00:09, 2259.50it/s]






 54%|█████▍    | 26428/48622 [00:11<00:09, 2256.59it/s]






 55%|█████▍    | 26654/48622 [00:11<00:10, 2183.02it/s]






 55%|█████▌    | 26873/48622 [00:11<00:10, 2159.64it/s]






 56%|█████▌    | 27096/48622 [00:11<00:09, 2178.96it/s]






 56%|███

 96%|█████████▌| 46738/48622 [00:20<00:00, 2269.42it/s]






 97%|█████████▋| 46969/48622 [00:20<00:00, 2279.66it/s]






 97%|█████████▋| 47198/48622 [00:20<00:00, 2265.59it/s]






 98%|█████████▊| 47428/48622 [00:20<00:00, 2273.86it/s]






 98%|█████████▊| 47656/48622 [00:20<00:00, 2266.26it/s]






 99%|█████████▊| 47893/48622 [00:20<00:00, 2294.46it/s]






 99%|█████████▉| 48123/48622 [00:20<00:00, 2257.83it/s]






 99%|█████████▉| 48350/48622 [00:20<00:00, 2235.61it/s]






100%|█████████▉| 48585/48622 [00:20<00:00, 2266.19it/s]






100%|██████████| 48622/48622 [00:20<00:00, 2316.47it/s]

In [651]:
## getting the sentences 
sentence = get_sentences(mix_sample,'tweetText')

In [653]:
# word 2 vec
model = Word2Vec(sentences, size=10,min_count=2)
w2v = dict(zip(model.wv.index2word, model.wv.syn0))

/Users/akshayuppal/anaconda3/envs/juul/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [654]:
# pickle the w2v word vectors  
with open("/Users/akshayuppal/Desktop/thesis/twitter_juul/models/w2v.pkl","wb") as f:
    pickle.dump(w2v,f)

## might as well user glove for word vectors and then take average of the vectors.. @TODO

In [655]:
#open the w2v model
w2v = pickle.load(open("/Users/akshayuppal/Desktop/thesis/twitter_juul/models/w2v.pkl","rb"))

In [659]:
w = 'legal'
model.wv.most_similar(positive=w)

/Users/akshayuppal/anaconda3/envs/juul/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('recreational', 0.9673788547515869),
 ('medical', 0.9437832832336426),
 ('cannabinoids', 0.941557765007019),
 ('legalize', 0.939355194568634),
 ('medicinal', 0.9389194846153259),
 ('permissive', 0.938389778137207),
 ('ocd', 0.9357061386108398),
 ('legalization', 0.9273738265037537),
 ('microdosing', 0.9165043234825134),
 ('vaporizing', 0.908244252204895)]

In [657]:
model.wv.similarity(w1='pot',w2='cannabis')

/Users/akshayuppal/anaconda3/envs/juul/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


0.8429552

In [418]:
w2v = dict(zip(model.wv.index2word, model.wv.syn0))

/Users/akshayuppal/anaconda3/envs/juul/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  """Entry point for launching an IPython kernel.


In [660]:
model.wv.similarity(w1='weed',w2='high')

/Users/akshayuppal/anaconda3/envs/juul/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


0.55198026

In [661]:
# class that returns word2vec
class MeanEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        # if a text is empty we should return a vector of zeros
        # with the same dimensionality as all the other vectors
        self.dim = len(next(iter(word2vec.values())))

    def fit(self, X, y):
        return self

    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec]
                    or [np.zeros(self.dim)], axis=0)
            for words in tqdm(X)
        ])

In [662]:
ob = MeanEmbeddingVectorizer(w2v)
w2v_features = ob.transform(sentences)








  0%|          | 0/48622 [00:00<?, ?it/s]






  4%|▍         | 1896/48622 [00:00<00:02, 18957.35it/s]






  8%|▊         | 3727/48622 [00:00<00:02, 18756.76it/s]






 12%|█▏        | 5610/48622 [00:00<00:02, 18778.56it/s]






 14%|█▍        | 6875/48622 [00:00<00:02, 16392.98it/s]






 18%|█▊        | 8720/48622 [00:00<00:02, 16958.67it/s]






 22%|██▏       | 10752/48622 [00:00<00:02, 17843.31it/s]






 26%|██▌       | 12737/48622 [00:00<00:01, 18399.59it/s]






 30%|███       | 14679/48622 [00:00<00:01, 18692.00it/s]






 34%|███▍      | 16664/48622 [00:00<00:01, 19024.19it/s]






 38%|███▊      | 18705/48622 [00:01<00:01, 19418.88it/s]






 43%|████▎     | 20687/48622 [00:01<00:01, 19534.96it/s]






 47%|████▋     | 22676/48622 [00:01<00:01, 19637.17it/s]






 51%|█████     | 24623/48622 [00:01<00:01, 19586.26it/s]






 55%|█████▍    | 26599/48622 [00:01<00:01, 19635.89it/s]






 59%|█████▉    | 28626/48622 [00:01<00:01, 19821.21it/s]






 63%|

In [663]:
df_input_data = mix_sample[['no_hashtags','no_urls','no_words','no_authors']]

In [664]:
## this is all of the data  ## can predit this..
input_data = np.hstack((df_input_data.values,w2v_features))

In [665]:
len(input_data)

48622

### Labelled_data

In [617]:
# so now to classify the text as substance abuse or not based on the tags..annotated
# so now will annotate the 500 sentences containing of weed words.. 
# going to label promotions and news as non weed
# labels (news: 1, research or study:2 , user_tweet: 3)

In [623]:
len(mix_sample)

48622

### dumnping file -- delete

In [669]:
# the data is mixed so taking a set of 500 tweets -- delete dumpinmg file
to_annotate = pd.DataFrame(mix_sample[401:500]['tweetText'])
to_annotate.to_csv("/Users/akshayuppal/Desktop/thesis/backup_stuff/to_annotate3.csv")

In [271]:
## laebelling it # labels (news: 1, research or study:2 , user_tweet: 3)...

## getting the labelled data

In [ ]:
pd.read_exce

In [319]:
df_weed_ann = pd.read_excel("/Users/akshayuppal/Desktop/thesis/backup_stuff/to_annotate_final.xlsx")
df_weed_ann.columns = ["tweetText","label"]
w2v = pickle.load(open("/Users/akshayuppal/Desktop/thesis/twitter_juul/models/w2v.pkl","rb"))

In [350]:
df_weed = (df_weed_ann[~df_weed_ann.isnull().any(axis=1)])

In [494]:
## getting sentences for labelled data
sentences_X = list()
for idx,row in df_weed.iterrows():
    words = get_tokenize_words(row['tweetText'])
    sentences_X.append(words)

In [551]:
## this is all of the data  ## can predit this..
ob = MeanEmbeddingVectorizer(w2v)
w2v_features_X = ob.transform(sentences_X)
# getting other features
df_temp = create_features(mix_sample[:len(df_weed)])
temp = df_temp.loc[: , df_temp.columns !='tweetText']
X = np.hstack((w2v_features_X,temp))








  0%|          | 0/119 [00:00<?, ?it/s]






100%|██████████| 119/119 [00:00<00:00, 2526.60it/s]

In [546]:
y = list(df_weed.label)

In [400]:
# lets get the accuracy for using the feature matrix as one returned by word2vec
from sklearn.pipeline import Pipeline
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.naive_bayes import GaussianNB

In [595]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn import svm
from sklearn.linear_model import LogisticRegression

# training naive bauyes on the data 500 sampled data

In [544]:
# normalize X
X = preprocessing.normalize(X, norm='l1')

In [559]:
NB = GaussianNB()

In [566]:
y = [int(i) for i in y]

In [575]:
clf = svm.SVC(kernel='linear', C=1)

In [592]:
etree = ExtraTreesClassifier(n_estimators=200)

In [598]:
Lr = LogisticRegression(random_state=0, solver='lbfgs',
                          multi_class='multinomial')

In [576]:
# try other models like svm , ...

In [603]:
score = cross_val_score(Lr,X,y)

In [604]:
score

array([0.43902439, 0.625     , 0.68421053])

In [605]:
# now tring with train_test split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size= 0.25,random_state =0)

In [606]:
Lr.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='multinomial',
          n_jobs=1, penalty='l2', random_state=0, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)

In [607]:
y_pred = Lr.predict(X_test)

In [608]:
y_pred_train = Lr.predict(X_train)

In [609]:
# train accuarcy
accuracy_score(y_train,y_pred=y_pred_train)

0.7640449438202247

In [610]:
# test accuracy
accuracy_score(y_test,y_pred=y_pred)

0.5666666666666667

In [445]:
## predicting all of the tweets
len(df_timeline)

1204733

In [493]:
sentences = get_sentences(df_timeline,'tweetText')






  0%|          | 0/1204733 [00:00<?, ?it/s]




  0%|          | 1/1204733 [00:00<43:07:48,  7.76it/s]




  0%|          | 203/1204733 [00:00<30:14:08, 11.07it/s]




  0%|          | 461/1204733 [00:00<21:11:57, 15.78it/s]




  0%|          | 654/1204733 [00:00<14:53:22, 22.46it/s]




  0%|          | 796/1204733 [00:00<10:30:24, 31.83it/s]




  0%|          | 1025/1204733 [00:00<7:23:50, 45.20it/s]




  0%|          | 1315/1204733 [00:00<5:12:41, 64.14it/s]




  0%|          | 1542/1204733 [00:00<3:41:30, 90.53it/s]




  0%|          | 1777/1204733 [00:00<2:37:34, 127.23it/s]




  0%|          | 2045/1204733 [00:01<1:52:32, 178.12it/s]




  0%|          | 2277/1204733 [00:01<1:21:41, 245.35it/s]




  0%|          | 2501/1204733 [00:01<1:00:27, 331.43it/s]




  0%|          | 2713/1204733 [00:01<45:42, 438.25it/s]  




  0%|          | 2914/1204733 [00:01<36:08, 554.33it/s]




  0%|          | 3099/1204733 [00:01<29:06, 687.92it/s]




  0%|          | 3337/1204733 [

  2%|▏         | 25712/1204733 [00:12<10:14, 1917.54it/s]




  2%|▏         | 25935/1204733 [00:12<09:49, 2000.25it/s]




  2%|▏         | 26198/1204733 [00:12<09:06, 2154.93it/s]




  2%|▏         | 26422/1204733 [00:12<09:59, 1964.30it/s]




  2%|▏         | 26648/1204733 [00:12<09:36, 2044.30it/s]




  2%|▏         | 26901/1204733 [00:12<09:03, 2167.05it/s]




  2%|▏         | 27125/1204733 [00:12<09:37, 2039.92it/s]




  2%|▏         | 27429/1204733 [00:12<08:40, 2261.97it/s]




  2%|▏         | 27668/1204733 [00:12<09:22, 2091.54it/s]




  2%|▏         | 27889/1204733 [00:13<09:31, 2059.70it/s]




  2%|▏         | 28149/1204733 [00:13<08:56, 2193.30it/s]




  2%|▏         | 28377/1204733 [00:13<08:52, 2207.51it/s]




  2%|▏         | 28656/1204733 [00:13<08:19, 2354.41it/s]




  2%|▏         | 28898/1204733 [00:13<09:13, 2122.55it/s]




  2%|▏         | 29120/1204733 [00:13<09:56, 1971.65it/s]




  2%|▏         | 29331/1204733 [00:13<09:44, 2010.78it/s]




  2%|▏  

  4%|▍         | 50700/1204733 [00:23<07:33, 2543.81it/s]




  4%|▍         | 50970/1204733 [00:23<07:55, 2427.27it/s]




  4%|▍         | 51225/1204733 [00:23<08:00, 2401.25it/s]




  4%|▍         | 51474/1204733 [00:24<08:03, 2385.76it/s]




  4%|▍         | 51719/1204733 [00:24<08:02, 2388.94it/s]




  4%|▍         | 51963/1204733 [00:24<08:33, 2243.61it/s]




  4%|▍         | 52213/1204733 [00:24<08:17, 2314.71it/s]




  4%|▍         | 52449/1204733 [00:24<08:15, 2323.76it/s]




  4%|▍         | 52724/1204733 [00:24<07:52, 2436.01it/s]




  4%|▍         | 52971/1204733 [00:24<08:06, 2366.61it/s]




  4%|▍         | 53211/1204733 [00:24<08:30, 2256.78it/s]




  4%|▍         | 53440/1204733 [00:24<08:36, 2227.96it/s]




  4%|▍         | 53665/1204733 [00:25<08:39, 2215.60it/s]




  4%|▍         | 53889/1204733 [00:25<09:20, 2053.66it/s]




  4%|▍         | 54098/1204733 [00:25<09:45, 1965.06it/s]




  5%|▍         | 54298/1204733 [00:25<10:06, 1898.11it/s]




  5%|▍  

  6%|▋         | 77089/1204733 [00:35<09:26, 1990.71it/s]




  6%|▋         | 77372/1204733 [00:35<08:36, 2182.08it/s]




  6%|▋         | 77629/1204733 [00:35<08:13, 2283.58it/s]




  6%|▋         | 77868/1204733 [00:35<09:04, 2068.77it/s]




  6%|▋         | 78155/1204733 [00:36<08:19, 2255.76it/s]




  7%|▋         | 78394/1204733 [00:36<08:43, 2151.29it/s]




  7%|▋         | 78712/1204733 [00:36<07:52, 2381.78it/s]




  7%|▋         | 78965/1204733 [00:36<07:51, 2386.44it/s]




  7%|▋         | 79226/1204733 [00:36<07:39, 2448.46it/s]




  7%|▋         | 79479/1204733 [00:36<08:56, 2098.07it/s]




  7%|▋         | 79703/1204733 [00:36<08:52, 2113.32it/s]




  7%|▋         | 79992/1204733 [00:36<08:09, 2298.13it/s]




  7%|▋         | 80292/1204733 [00:36<07:35, 2470.45it/s]




  7%|▋         | 80557/1204733 [00:37<07:26, 2519.18it/s]




  7%|▋         | 80818/1204733 [00:37<08:03, 2322.18it/s]




  7%|▋         | 81060/1204733 [00:37<08:16, 2263.89it/s]




  7%|▋  

  9%|▊         | 104064/1204733 [00:47<06:47, 2698.74it/s]




  9%|▊         | 104344/1204733 [00:47<07:11, 2550.06it/s]




  9%|▊         | 104669/1204733 [00:47<06:43, 2725.45it/s]




  9%|▊         | 104951/1204733 [00:47<08:20, 2198.78it/s]




  9%|▊         | 105194/1204733 [00:47<08:08, 2250.69it/s]




  9%|▉         | 105436/1204733 [00:47<08:52, 2064.81it/s]




  9%|▉         | 105698/1204733 [00:48<08:18, 2203.52it/s]




  9%|▉         | 105931/1204733 [00:48<08:38, 2120.99it/s]




  9%|▉         | 106153/1204733 [00:48<09:07, 2007.01it/s]




  9%|▉         | 106418/1204733 [00:48<08:27, 2163.94it/s]




  9%|▉         | 106646/1204733 [00:48<08:19, 2197.14it/s]




  9%|▉         | 106873/1204733 [00:48<08:16, 2209.92it/s]




  9%|▉         | 107106/1204733 [00:48<08:09, 2243.94it/s]




  9%|▉         | 107376/1204733 [00:48<07:44, 2363.52it/s]




  9%|▉         | 107617/1204733 [00:48<07:54, 2314.03it/s]




  9%|▉         | 107852/1204733 [00:49<07:53, 2317.93it

...

 91%|█████████ | 1097735/1204733 [08:27<00:44, 2400.66it/s]




 91%|█████████ | 1098017/1204733 [08:27<00:42, 2511.40it/s]




 91%|█████████ | 1098272/1204733 [08:27<00:47, 2222.98it/s]




 91%|█████████ | 1098577/1204733 [08:27<00:43, 2417.84it/s]




 91%|█████████ | 1098853/1204733 [08:28<00:42, 2508.16it/s]




 91%|█████████ | 1099113/1204733 [08:28<00:44, 2384.72it/s]




 91%|█████████▏| 1099359/1204733 [08:28<00:45, 2327.32it/s]




 91%|█████████▏| 1099614/1204733 [08:28<00:44, 2388.09it/s]




 91%|█████████▏| 1099871/1204733 [08:28<00:42, 2439.58it/s]




 91%|█████████▏| 1100292/1204733 [08:28<00:37, 2791.68it/s]




 91%|█████████▏| 1100592/1204733 [08:28<00:40, 2566.46it/s]




 91%|█████████▏| 1100867/1204733 [08:28<00:40, 2574.15it/s]




 91%|█████████▏| 1101137/1204733 [08:28<00:41, 2468.49it/s]




 91%|█████████▏| 1101413/1204733 [08:29<00:40, 2548.64it/s]




 91%|█████████▏| 1101726/1204733 [08:29<00:38, 2698.83it/s]




 91%|█████████▏| 1102004/1204733 [08:29<

 93%|█████████▎| 1123908/1204733 [08:39<00:33, 2416.73it/s]




 93%|█████████▎| 1124162/1204733 [08:39<00:33, 2402.18it/s]




 93%|█████████▎| 1124411/1204733 [08:39<00:34, 2347.29it/s]




 93%|█████████▎| 1124653/1204733 [08:39<00:33, 2368.12it/s]




 93%|█████████▎| 1124895/1204733 [08:39<00:35, 2270.94it/s]




 93%|█████████▎| 1125126/1204733 [08:39<00:36, 2164.34it/s]




 93%|█████████▎| 1125347/1204733 [08:39<00:37, 2131.81it/s]




 93%|█████████▎| 1125674/1204733 [08:40<00:33, 2378.42it/s]




 93%|█████████▎| 1125981/1204733 [08:40<00:30, 2549.30it/s]




 93%|█████████▎| 1126280/1204733 [08:40<00:29, 2665.58it/s]




 94%|█████████▎| 1126556/1204733 [08:40<00:33, 2363.67it/s]




 94%|█████████▎| 1126806/1204733 [08:40<00:36, 2116.98it/s]




 94%|█████████▎| 1127032/1204733 [08:40<00:38, 2027.93it/s]




 94%|█████████▎| 1127327/1204733 [08:40<00:34, 2236.91it/s]




 94%|█████████▎| 1127575/1204733 [08:40<00:33, 2302.30it/s]




 94%|█████████▎| 1127816/1204733 [08:40<

 95%|█████████▌| 1149289/1204733 [08:50<00:27, 1982.15it/s]




 95%|█████████▌| 1149494/1204733 [08:50<00:29, 1865.34it/s]




 95%|█████████▌| 1149687/1204733 [08:51<00:32, 1719.66it/s]




 95%|█████████▌| 1149866/1204733 [08:51<00:33, 1645.31it/s]




 95%|█████████▌| 1150068/1204733 [08:51<00:31, 1741.11it/s]




 95%|█████████▌| 1150248/1204733 [08:51<00:33, 1629.97it/s]




 95%|█████████▌| 1150417/1204733 [08:51<00:33, 1637.01it/s]




 96%|█████████▌| 1150740/1204733 [08:51<00:28, 1919.66it/s]




 96%|█████████▌| 1151039/1204733 [08:51<00:24, 2149.98it/s]




 96%|█████████▌| 1151300/1204733 [08:51<00:23, 2269.11it/s]




 96%|█████████▌| 1151574/1204733 [08:51<00:22, 2391.33it/s]




 96%|█████████▌| 1151828/1204733 [08:52<00:24, 2125.78it/s]




 96%|█████████▌| 1152057/1204733 [08:52<00:26, 1981.67it/s]




 96%|█████████▌| 1152269/1204733 [08:52<00:26, 2010.69it/s]




 96%|█████████▌| 1152480/1204733 [08:52<00:26, 1951.35it/s]




 96%|█████████▌| 1152693/1204733 [08:52<

 97%|█████████▋| 1174190/1204733 [09:02<00:16, 1847.38it/s]




 97%|█████████▋| 1174440/1204733 [09:02<00:15, 2004.09it/s]




 98%|█████████▊| 1174746/1204733 [09:02<00:13, 2235.06it/s]




 98%|█████████▊| 1175064/1204733 [09:02<00:12, 2453.45it/s]




 98%|█████████▊| 1175330/1204733 [09:02<00:11, 2511.22it/s]




 98%|█████████▊| 1175593/1204733 [09:03<00:11, 2450.93it/s]




 98%|█████████▊| 1175847/1204733 [09:03<00:11, 2421.99it/s]




 98%|█████████▊| 1176096/1204733 [09:03<00:12, 2246.82it/s]




 98%|█████████▊| 1176357/1204733 [09:03<00:12, 2343.90it/s]




 98%|█████████▊| 1176626/1204733 [09:03<00:11, 2436.65it/s]




 98%|█████████▊| 1176899/1204733 [09:03<00:11, 2515.62it/s]




 98%|█████████▊| 1177155/1204733 [09:03<00:11, 2407.75it/s]




 98%|█████████▊| 1177425/1204733 [09:03<00:10, 2488.13it/s]




 98%|█████████▊| 1177678/1204733 [09:03<00:11, 2397.88it/s]




 98%|█████████▊| 1177951/1204733 [09:03<00:10, 2488.28it/s]




 98%|█████████▊| 1178203/1204733 [09:04<

100%|█████████▉| 1200856/1204733 [09:13<00:01, 2184.04it/s]




100%|█████████▉| 1201080/1204733 [09:14<00:01, 2173.35it/s]




100%|█████████▉| 1201391/1204733 [09:14<00:01, 2388.16it/s]




100%|█████████▉| 1201639/1204733 [09:14<00:01, 2156.27it/s]




100%|█████████▉| 1201866/1204733 [09:14<00:01, 2021.17it/s]




100%|█████████▉| 1202078/1204733 [09:14<00:01, 1693.37it/s]




100%|█████████▉| 1202263/1204733 [09:14<00:01, 1639.70it/s]




100%|█████████▉| 1202486/1204733 [09:14<00:01, 1780.09it/s]




100%|█████████▉| 1202676/1204733 [09:14<00:01, 1641.42it/s]




100%|█████████▉| 1202888/1204733 [09:15<00:01, 1760.58it/s]




100%|█████████▉| 1203074/1204733 [09:15<00:00, 1727.24it/s]




100%|█████████▉| 1203264/1204733 [09:15<00:00, 1772.83it/s]




100%|█████████▉| 1203447/1204733 [09:15<00:00, 1489.67it/s]




100%|█████████▉| 1203677/1204733 [09:15<00:00, 1664.95it/s]




100%|█████████▉| 1203886/1204733 [09:15<00:00, 1772.73it/s]




100%|█████████▉| 1204147/1204733 [09:15<

In [497]:
## this is all of the data  ## can predit this..
ob = MeanEmbeddingVectorizer(w2v)
w2v_features_X = ob.transform(sentences)
# getting other features
df_temp = create_features(df_timeline)
temp = df_temp.loc[: , df_temp.columns !='tweetText']
X = np.hstack((w2v_features_X,temp))






  0%|          | 0/1204733 [00:00<?, ?it/s]




  0%|          | 1/1204733 [00:00<40:49:40,  8.20it/s]




  0%|          | 203/1204733 [00:00<28:37:28, 11.69it/s]




  0%|          | 456/1204733 [00:00<20:04:21, 16.67it/s]




  0%|          | 642/1204733 [00:00<14:06:10, 23.72it/s]




  0%|          | 781/1204733 [00:00<9:57:12, 33.60it/s] 




  0%|          | 979/1204733 [00:00<7:01:01, 47.65it/s]




  0%|          | 1270/1204733 [00:00<4:56:42, 67.60it/s]




  0%|          | 1503/1204733 [00:00<3:30:15, 95.37it/s]




  0%|          | 1730/1204733 [00:00<2:29:49, 133.82it/s]




  0%|          | 1983/1204733 [00:01<1:47:14, 186.91it/s]




  0%|          | 2207/1204733 [00:01<1:17:47, 257.66it/s]




  0%|          | 2430/1204733 [00:01<57:42, 347.20it/s]  




  0%|          | 2641/1204733 [00:01<43:33, 459.98it/s]




  0%|          | 2846/1204733 [00:01<34:49, 575.10it/s]




  0%|          | 3031/1204733 [00:01<28:39, 698.70it/s]




  0%|          | 3211/1204733 [00:

  2%|▏         | 25655/1204733 [00:12<10:11, 1929.43it/s]




  2%|▏         | 25861/1204733 [00:12<10:06, 1945.19it/s]




  2%|▏         | 26147/1204733 [00:12<09:07, 2150.93it/s]




  2%|▏         | 26376/1204733 [00:12<09:30, 2065.53it/s]




  2%|▏         | 26593/1204733 [00:12<09:45, 2012.47it/s]




  2%|▏         | 26852/1204733 [00:12<09:06, 2155.68it/s]




  2%|▏         | 27076/1204733 [00:12<09:37, 2040.22it/s]




  2%|▏         | 27377/1204733 [00:12<08:41, 2258.36it/s]




  2%|▏         | 27615/1204733 [00:13<08:51, 2215.26it/s]




  2%|▏         | 27846/1204733 [00:13<09:43, 2018.30it/s]




  2%|▏         | 28115/1204733 [00:13<09:00, 2178.84it/s]




  2%|▏         | 28344/1204733 [00:13<09:09, 2141.70it/s]




  2%|▏         | 28632/1204733 [00:13<08:27, 2318.48it/s]




  2%|▏         | 28874/1204733 [00:13<09:13, 2123.63it/s]




  2%|▏         | 29097/1204733 [00:13<09:52, 1982.63it/s]




  2%|▏         | 29321/1204733 [00:13<09:33, 2051.04it/s]




  2%|▏  

  4%|▍         | 50746/1204733 [00:23<07:42, 2493.67it/s]




  4%|▍         | 51011/1204733 [00:23<07:50, 2451.44it/s]




  4%|▍         | 51267/1204733 [00:24<08:06, 2372.96it/s]




  4%|▍         | 51513/1204733 [00:24<08:03, 2385.43it/s]




  4%|▍         | 51765/1204733 [00:24<07:55, 2423.73it/s]




  4%|▍         | 52012/1204733 [00:24<08:26, 2277.32it/s]




  4%|▍         | 52264/1204733 [00:24<08:11, 2342.77it/s]




  4%|▍         | 52502/1204733 [00:24<08:22, 2294.37it/s]




  4%|▍         | 52793/1204733 [00:24<07:50, 2447.19it/s]




  4%|▍         | 53043/1204733 [00:24<08:22, 2290.26it/s]




  4%|▍         | 53292/1204733 [00:24<08:10, 2345.34it/s]




  4%|▍         | 53531/1204733 [00:25<08:42, 2202.05it/s]




  4%|▍         | 53756/1204733 [00:25<09:05, 2110.46it/s]




  4%|▍         | 53972/1204733 [00:25<09:13, 2077.68it/s]




  4%|▍         | 54183/1204733 [00:25<09:37, 1993.06it/s]




  5%|▍         | 54386/1204733 [00:25<09:54, 1934.91it/s]




  5%|▍  

  6%|▋         | 76657/1204733 [00:35<09:49, 1913.90it/s]




  6%|▋         | 76889/1204733 [00:35<09:18, 2019.03it/s]




  6%|▋         | 77098/1204733 [00:36<10:01, 1874.01it/s]




  6%|▋         | 77371/1204733 [00:36<09:06, 2063.33it/s]




  6%|▋         | 77616/1204733 [00:36<08:40, 2165.20it/s]




  6%|▋         | 77842/1204733 [00:36<09:40, 1941.51it/s]




  6%|▋         | 78128/1204733 [00:36<08:44, 2146.79it/s]




  7%|▋         | 78357/1204733 [00:36<09:28, 1980.95it/s]




  7%|▋         | 78669/1204733 [00:36<08:26, 2224.45it/s]




  7%|▋         | 78914/1204733 [00:36<08:12, 2285.80it/s]




  7%|▋         | 79156/1204733 [00:36<08:13, 2280.92it/s]




  7%|▋         | 79394/1204733 [00:37<08:49, 2124.29it/s]




  7%|▋         | 79616/1204733 [00:37<09:33, 1960.57it/s]




  7%|▋         | 79899/1204733 [00:37<08:40, 2159.48it/s]




  7%|▋         | 80187/1204733 [00:37<08:02, 2332.92it/s]




  7%|▋         | 80446/1204733 [00:37<07:47, 2403.62it/s]




  7%|▋  

  9%|▊         | 102464/1204733 [00:47<07:48, 2351.08it/s]




  9%|▊         | 102706/1204733 [00:47<08:07, 2261.48it/s]




  9%|▊         | 102939/1204733 [00:47<08:03, 2279.64it/s]




  9%|▊         | 103171/1204733 [00:47<08:31, 2154.52it/s]




  9%|▊         | 103491/1204733 [00:48<07:41, 2388.48it/s]




  9%|▊         | 103777/1204733 [00:48<07:18, 2511.47it/s]




  9%|▊         | 104090/1204733 [00:48<06:52, 2667.34it/s]




  9%|▊         | 104366/1204733 [00:48<07:26, 2464.96it/s]




  9%|▊         | 104690/1204733 [00:48<06:54, 2651.00it/s]




  9%|▊         | 104966/1204733 [00:48<08:30, 2155.04it/s]




  9%|▊         | 105204/1204733 [00:48<08:15, 2217.87it/s]




  9%|▉         | 105442/1204733 [00:48<09:03, 2022.55it/s]




  9%|▉         | 105713/1204733 [00:49<08:22, 2188.28it/s]




  9%|▉         | 105947/1204733 [00:49<08:42, 2104.58it/s]




  9%|▉         | 106169/1204733 [00:49<09:13, 1985.57it/s]




  9%|▉         | 106444/1204733 [00:49<08:27, 2165.10it

...

 93%|█████████▎| 1120341/1204733 [08:26<00:40, 2089.53it/s]




 93%|█████████▎| 1120555/1204733 [08:26<00:43, 1935.89it/s]




 93%|█████████▎| 1120754/1204733 [08:27<00:48, 1735.84it/s]




 93%|█████████▎| 1120936/1204733 [08:27<00:49, 1709.05it/s]




 93%|█████████▎| 1121202/1204733 [08:27<00:43, 1913.99it/s]




 93%|█████████▎| 1121496/1204733 [08:27<00:38, 2137.37it/s]




 93%|█████████▎| 1121727/1204733 [08:27<00:38, 2139.82it/s]




 93%|█████████▎| 1121953/1204733 [08:27<00:40, 2030.52it/s]




 93%|█████████▎| 1122166/1204733 [08:27<00:42, 1927.49it/s]




 93%|█████████▎| 1122446/1204733 [08:27<00:38, 2125.66it/s]




 93%|█████████▎| 1122681/1204733 [08:27<00:37, 2188.10it/s]




 93%|█████████▎| 1122944/1204733 [08:28<00:35, 2304.00it/s]




 93%|█████████▎| 1123241/1204733 [08:28<00:33, 2469.32it/s]




 93%|█████████▎| 1123497/1204733 [08:28<00:36, 2204.69it/s]




 93%|█████████▎| 1123749/1204733 [08:28<00:35, 2290.04it/s]




 93%|█████████▎| 1124016/1204733 [08:28<

 95%|█████████▌| 1145769/1204733 [08:38<00:28, 2104.07it/s]




 95%|█████████▌| 1145991/1204733 [08:38<00:28, 2031.78it/s]




 95%|█████████▌| 1146257/1204733 [08:38<00:26, 2186.06it/s]




 95%|█████████▌| 1146485/1204733 [08:38<00:28, 2032.68it/s]




 95%|█████████▌| 1146771/1204733 [08:38<00:26, 2223.66it/s]




 95%|█████████▌| 1147005/1204733 [08:38<00:26, 2140.48it/s]




 95%|█████████▌| 1147330/1204733 [08:39<00:24, 2383.20it/s]




 95%|█████████▌| 1147584/1204733 [08:39<00:23, 2393.36it/s]




 95%|█████████▌| 1147834/1204733 [08:39<00:23, 2375.99it/s]




 95%|█████████▌| 1148079/1204733 [08:39<00:23, 2390.57it/s]




 95%|█████████▌| 1148324/1204733 [08:39<00:24, 2272.43it/s]




 95%|█████████▌| 1148572/1204733 [08:39<00:24, 2329.81it/s]




 95%|█████████▌| 1148809/1204733 [08:39<00:26, 2127.99it/s]




 95%|█████████▌| 1149052/1204733 [08:39<00:25, 2210.34it/s]




 95%|█████████▌| 1149279/1204733 [08:39<00:27, 1982.13it/s]




 95%|█████████▌| 1149486/1204733 [08:40<

 97%|█████████▋| 1171530/1204733 [08:50<00:15, 2128.00it/s]




 97%|█████████▋| 1171782/1204733 [08:50<00:14, 2231.77it/s]




 97%|█████████▋| 1172113/1204733 [08:50<00:13, 2471.64it/s]




 97%|█████████▋| 1172371/1204733 [08:50<00:15, 2078.88it/s]




 97%|█████████▋| 1172597/1204733 [08:50<00:16, 1997.76it/s]




 97%|█████████▋| 1172829/1204733 [08:50<00:15, 2084.30it/s]




 97%|█████████▋| 1173095/1204733 [08:50<00:14, 2228.63it/s]




 97%|█████████▋| 1173328/1204733 [08:50<00:13, 2244.19it/s]




 97%|█████████▋| 1173560/1204733 [08:50<00:14, 2197.97it/s]




 97%|█████████▋| 1173785/1204733 [08:51<00:13, 2213.09it/s]




 97%|█████████▋| 1174010/1204733 [08:51<00:16, 1900.85it/s]




 97%|█████████▋| 1174211/1204733 [08:51<00:15, 1913.89it/s]




 97%|█████████▋| 1174488/1204733 [08:51<00:14, 2109.30it/s]




 98%|█████████▊| 1174814/1204733 [08:51<00:12, 2356.98it/s]




 98%|█████████▊| 1175125/1204733 [08:51<00:11, 2540.03it/s]




 98%|█████████▊| 1175440/1204733 [08:51<

100%|█████████▉| 1199121/1204733 [09:01<00:02, 1947.63it/s]




100%|█████████▉| 1199367/1204733 [09:01<00:02, 2076.38it/s]




100%|█████████▉| 1199625/1204733 [09:01<00:02, 2205.01it/s]




100%|█████████▉| 1199853/1204733 [09:01<00:02, 2024.42it/s]




100%|█████████▉| 1200078/1204733 [09:02<00:02, 2086.83it/s]




100%|█████████▉| 1200293/1204733 [09:02<00:02, 1992.06it/s]




100%|█████████▉| 1200498/1204733 [09:02<00:02, 2003.90it/s]




100%|█████████▉| 1200807/1204733 [09:02<00:01, 2236.15it/s]




100%|█████████▉| 1201042/1204733 [09:02<00:01, 2074.27it/s]




100%|█████████▉| 1201375/1204733 [09:02<00:01, 2336.16it/s]




100%|█████████▉| 1201627/1204733 [09:02<00:01, 2201.04it/s]




100%|█████████▉| 1201862/1204733 [09:02<00:01, 2044.08it/s]




100%|█████████▉| 1202079/1204733 [09:03<00:01, 1728.28it/s]




100%|█████████▉| 1202269/1204733 [09:03<00:01, 1705.80it/s]




100%|█████████▉| 1202498/1204733 [09:03<00:01, 1845.70it/s]




100%|█████████▉| 1202695/1204733 [09:03<

ValueError: all the input array dimensions except for the concatenation axis must match exactly

In [533]:
# so now based on tweet we can label the user as poly and mono finally.. 


In [613]:
users = df_timeline.userID.unique()
for user in users:
    df_user = df_timeline.loc[df_timeline.userID == user]
    sentences = get_sentences(df_user,'tweetText')
    w2v_features_X = ob.transform(sentences)
    # getting other features
    df_temp = create_features(df_user)
    temp = df_temp.loc[: , df_temp.columns !='tweetText']
    X = np.hstack((w2v_features_X,temp))
    # predicting the sentences
    y_pred = Lr.predict(X)
    break








  0%|          | 0/40 [00:00<?, ?it/s]






100%|██████████| 40/40 [00:00<00:00, 3252.60it/s]






  0%|          | 0/40 [00:00<?, ?it/s]






100%|██████████| 40/40 [00:00<00:00, 3146.10it/s]

In [615]:
# predicting the sentences
y_pred = Lr.predict(X)

In [616]:
y_pred

array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 1, 3, 3, 3, 3, 3, 3, 3, 2, 3, 3, 3, 3, 3, 3, 3])